In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("/Users/piyush/projects/ViDA-SSL/")

In [52]:
from os.path import join

import numpy as np

from utils.io import load_json, load_txt

In [53]:
annot_path = "/Users/piyush/projects/ViDA-SSL/data/FineGym/annotations"\
    "/action-temporal-localization/finegym_annotation_info_v1.0.json"
train_split_file = "/Users/piyush/projects/ViDA-SSL/data/FineGym/splits/gym99_train.txt"
val_split_file = "/Users/piyush/projects/ViDA-SSL/data/FineGym/splits/gym99_val.txt"

In [6]:
annot = load_json(annot_path)

In [8]:
len(annot)

303

In [40]:
events = []
stages = []
segment_times = []

subactions = dict()

for vkey in annot:
    edict = annot[vkey]
    for ekey in edict:
        events.append(annot[vkey][ekey]["event"])

        segments = edict[ekey]["segments"]
        if segments is not None:
            for skey in segments:
                stages.append(segments[skey]["stages"])
                
                tstamps = segments[skey]["timestamps"]
                segment_times.extend([x[1] - x[0] for x in tstamps])

In [41]:
len(segment_times)

36764

In [43]:
np.mean(segment_times)

1.545762974649113

In [47]:
subaction_times = dict()

video_keys = annot.keys()

for vkey in video_keys:
    
    event_keys = annot[vkey].keys()
    
    for ekey in event_keys:
        
        subact_dict = annot[vkey][ekey]["segments"]
        if subact_dict is not None:
            for skey, sval in subact_dict.items():
                instance_key = f"{vkey}_{ekey}_{skey}"
                subaction_times[instance_key] = sval["timestamps"]

In [49]:
len(subaction_times)

32697

In [50]:
list(subaction_times.keys())[:10]

['0LtLS9wROrk_E_002407_002435_A_0003_0005',
 '0LtLS9wROrk_E_002407_002435_A_0006_0008',
 '0LtLS9wROrk_E_002407_002435_A_0012_0013',
 '0LtLS9wROrk_E_002407_002435_A_0015_0017',
 '0LtLS9wROrk_E_002407_002435_A_0019_0021',
 '0LtLS9wROrk_E_002407_002435_A_0021_0023',
 '0LtLS9wROrk_E_002407_002435_A_0023_0028',
 '0LtLS9wROrk_E_002451_002529_A_0019_0021',
 '0LtLS9wROrk_E_002451_002529_A_0022_0023',
 '0LtLS9wROrk_E_002451_002529_A_0033_0034']

In [51]:
subaction_times["0LtLS9wROrk_E_002407_002435_A_0015_0017"]

[[15.59, 17.27]]

### Load split files

In [56]:
train_split = load_txt(train_split_file)
train_ids = [x.split()[0] for x in train_split]

val_split = load_txt(val_split_file)
val_ids = [x.split()[0] for x in val_split]

In [57]:
len(train_ids), len(val_ids)

(20484, 8521)

In [60]:
train_ids[0] in subaction_times, \
val_ids[0] in subaction_times

(True, True)

In [61]:
split_ids = train_ids + val_ids

### Filter based on given split

In [62]:
split_subaction_times = {k:v for k, v in subaction_times.items() if k in split_ids}

In [63]:
len(split_subaction_times)

29005

In [66]:
total_times = []
for key, value in split_subaction_times.items():
    total_times.extend([x[1] - x[0] for x in value])

In [67]:
len(total_times)

32045

In [69]:
np.mean(total_times)

1.5071196754563894